# HPPC

Segment a HPPC profile into pulses and save them to individual files.

In [ ]:
import bdat
import pandas as pd
import altair as alt
from IPython.display import display

Import the data

In [ ]:
# build metadata
batterytype = bdat.BatterySpecies("BAK CG50", capacity=5.0, endOfChargeVoltage=4.2, endOfDischargeVoltage=2.5)
battery = bdat.Battery("DigiBatt_CG50_1072", type=batterytype)
cycling = bdat.Cycling("checkup example", object=battery)

df = pd.read_parquet("DigiBatt_CG50_1173_HPPC.parquet")
dataspec = bdat.get_dataspec(cycling, df)
data = bdat.CyclingData(cycling, df, dataspec)

Segment cycling data into steps

In [ ]:
# find steps
steplist = bdat.steps.find_steps(data)

# plot steps
bdat.plots.plot_steps(None, steplist, data.df, dataspec).show()

Find pulses in the test steps

In [ ]:
# specify patterns to look for
patterntypes = [bdat.patterns.Pulse()]

# search the test steps for matches
testeval = bdat.patterns.find_patterns(steplist, patterntypes, data)

# plot results
bdat.plots.plot_testevals(None, testeval, data.df).show()

Save a file for each pulse. Each file starts 30 seconds before the pulse and ends 30 seconds after it.

In [ ]:
for i, eval in enumerate(testeval.evals):
    data.df[(data.time > (eval.start - 30)) & (data.time < (eval.end + 30))].to_csv(f"DigiBatt_CG50_1173_HPPC_Pulse_{i}.csv", index=False)